In [1]:
import pandas as pd

In [2]:
## Set-up
import requests
import json

# Insert activated key
key = '1b5678a1a374e17f9c32c8981fbe81d8f437edc7'

In [3]:
## Specify variables for API call
pop_variables = ['P2_001N', # Total
                 'P2_002N', # Total Hipsanic or Latino
                 'P2_003N', # Total not Hispanic or Latino
                 'P2_004N', # Total not Hispanic or Latino, population of one race
                 'P2_005N', # Total not Hispanic or Latino, population of one race, White alone
                 'P2_006N', # Total not Hispanic or Latino, population of one race, Black alone
                 'P2_007N', # Total not Hispanic or Latino, population of one race, American Indian and Alaska Native alone
                 'P2_008N', # Total not Hispanic or Latino, population of one race, Asian alone
                 'P2_009N', # Total not Hispanic or Latino, population of one race, Native Hawaiian and Other Pacific Islander alone
                 'P2_010N', # Total not Hispanic or Latino, population of one race, some other race alone
                 'P2_011N'] # Total not Hispanic or Latino, population of two or more races

# Concatenate into single string to feed into url
pop_variables_str = ','.join(pop_variables)

In [4]:
def get_census_pop_data(year):
    ## Specify URL, inserting population variables and activation key
    url = f'https://api.census.gov/data/{year}/dec/pl?get=NAME,{pop_variables_str}&for=county:*&key={key}'

    ## Store response
    data = requests.get(url)

    ## Check response
    if data.status_code == 200:
        print('Status code:', data.status_code, '- OK')
    else:
        print('Status code:', data.status_code, '- ERROR')
        
    ## Store API response in a variable
    available_data = data.json()

    ## Create dataframe
    census2020_pop_race = pd.DataFrame(available_data[1:], columns = available_data[0])
    census2020_pop_race['FIPS'] = census2020_pop_race['state'] + census2020_pop_race['county']
    census2020_pop_race = census2020_pop_race.drop(columns = ['state', 'county'])
    for var in pop_variables:
        census2020_pop_race[var] = pd.to_numeric(census2020_pop_race[var])
    
    ## Return dataframe
    return census2020_pop_race